# Get damping derivatives by adding additional velocities into v_inf

In [14]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from src import Wing, FlightCondition, WingPool, Simulation, PostProcessing, load_folder

airfoils_data, _ = load_folder("sample_airfoils")
# print(airfoils_data)

wing = Wing(
    spans=[1, 1],
    chords=[1, 1, 1],
    offsets=[0, 0, 0],
    twist_angles=[0, 0, 0],
    dihedral_angles=[0, 0, 0],
    airfoils=["NACA4424", "NACA4424", "NACA4412"],
    surface_name="wing",
    N_panels=4,
    distribution_type="cosine",
    sweep_check=False,
)
wing.generate_mesh()  # Generate wing simulation elements

eh = Wing(
    spans=[1, 1],
    chords=[1, 1, 1],
    offsets=[0, 0, 0],
    twist_angles=[0, 0, 0],
    dihedral_angles=[0, 0, 0],
    airfoils=["NACA4424", "NACA4424", "NACA4412"],
    surface_name="eh",
    N_panels=4,
    distribution_type="cosine",
    sweep_check=False,
    x_pos=5
)
eh.generate_mesh()  # Generate wing simulation elements

flight_condition = FlightCondition(
    V_inf=20,
    nu=1.5e-5,
    rho=1.225,
    angles_of_attack=[alpha for alpha in range(2)],
    h=0,
    ground_effect_check=False,
    angular_velocity=[0, 0.1, 0]
)
wing.setup_airfoil_data(flight_condition, airfoils_data)  # Assign airfoil data to wing
eh.setup_airfoil_data(flight_condition, airfoils_data)

# Link wing that will be simulated with flight condition
wing_pool = WingPool(wing_list=[wing, eh], flight_condition=flight_condition, moment_ref=[0.5, 0, 0])

In [15]:
import pprint
import numpy as np

v_inf_array = flight_condition.v_inf_list[0]
moment_ref = wing_pool.system_moment_ref
# print(moment_ref)
for k, v in wing_pool.system_freestream_velocities.items():
    pprint.pp(v)

print(wing_pool.get_angular_velocity(np.array([0, 1, 0])))

{'wing': array([[1.   , 0.   , 0.025],
       [1.   , 0.   , 0.025],
       [1.   , 0.   , 0.025],
       [1.   , 0.   , 0.025]]),
 'wing_mirrored': array([[1.   , 0.   , 0.025],
       [1.   , 0.   , 0.025],
       [1.   , 0.   , 0.025],
       [1.   , 0.   , 0.025]]),
 'eh': array([[ 1.   ,  0.   , -0.475],
       [ 1.   ,  0.   , -0.475],
       [ 1.   ,  0.   , -0.475],
       [ 1.   ,  0.   , -0.475]]),
 'eh_mirrored': array([[ 1.   ,  0.   , -0.475],
       [ 1.   ,  0.   , -0.475],
       [ 1.   ,  0.   , -0.475],
       [ 1.   ,  0.   , -0.475]])}
{'wing': array([[0.9998477 , 0.        , 0.04245241],
       [0.9998477 , 0.        , 0.04245241],
       [0.9998477 , 0.        , 0.04245241],
       [0.9998477 , 0.        , 0.04245241]]),
 'wing_mirrored': array([[0.9998477 , 0.        , 0.04245241],
       [0.9998477 , 0.        , 0.04245241],
       [0.9998477 , 0.        , 0.04245241],
       [0.9998477 , 0.        , 0.04245241]]),
 'eh': array([[ 0.9998477 ,  0.        , -0.457

In [6]:
import numpy as np
import numpy.linalg as npla

ref = moment_ref["wing"]
angular_vector = np.array([0, 1, 0])[np.newaxis, :]

print(ref.shape)
print(angular_vector.shape)

angular_velocity = np.cross(ref, angular_vector)

print(ref)
print(angular_velocity)

print("for loop")
for i in ref:
    print(np.cross(i, angular_vector))

print(angular_vector)

(4, 3)
(1, 3)
[[-2.73535534  0.14644661  0.        ]
 [-2.66464466  0.85355339  0.        ]
 [-2.63535534  1.14644661  0.        ]
 [-2.56464466  1.85355339  0.        ]]
[[ 0.          0.         -2.73535534]
 [ 0.          0.         -2.66464466]
 [ 0.          0.         -2.63535534]
 [ 0.          0.         -2.56464466]]
for loop
[[ 0.          0.         -2.73535534]]
[[ 0.          0.         -2.66464466]]
[[ 0.          0.         -2.63535534]]
[[ 0.          0.         -2.56464466]]
[[0 1 0]]


In [7]:
import numpy as np
import numpy.linalg as npla

# following the relation V = w * r
def get_angular_velocity(system_moment_ref, angular_vector) -> np.ndarray:
    system_angular_velocities = {}
    for wing, ref in system_moment_ref.items():
        system_angular_velocities[wing] = np.cross(angular_vector, ref)
    return system_angular_velocities

ref = moment_ref["wing"]
angular_vector = np.array([0, 1, 0])[np.newaxis, :]

system_angular_velocities = get_angular_velocity(moment_ref, angular_vector)
for wing, velocities in system_angular_velocities.items():
    print(f'{wing}: \n{velocities}\n\n')

wing: 
[[ 0.         -0.          2.73535534]
 [ 0.         -0.          2.66464466]
 [ 0.         -0.          2.63535534]
 [ 0.         -0.          2.56464466]]


wing_mirrored: 
[[ 0.         -0.          2.73535534]
 [ 0.         -0.          2.66464466]
 [ 0.         -0.          2.63535534]
 [ 0.         -0.          2.56464466]]


eh: 
[[ 0.          0.         -2.26464466]
 [ 0.          0.         -2.33535534]
 [ 0.          0.         -2.36464466]
 [ 0.          0.         -2.43535534]]


eh_mirrored: 
[[ 0.          0.         -2.26464466]
 [ 0.          0.         -2.33535534]
 [ 0.          0.         -2.36464466]
 [ 0.          0.         -2.43535534]]


